# 合成資料模型訓練模組 - python3.9 版本

## 環境建置

Python 3.9.23

1. 建立 virtualenv
```
python -m venv venv
source venv/bin/activate
```
2. 安裝 core 套件（預先裝 wheel）
```
pip install --upgrade pip setuptools wheel
# pip install -r requirements.txt
pip install sdv==1.23.0
pip install sdmetrics==0.21.0
pip install pandas==2.3.0
pip install numpy==2.0.2
pip install scikit-learn==1.6.1
pip install scipy==1.13.1
pip install matplotlib==3.9.4

```

In [37]:
import pandas as pd
import logging
import time
import os
from sdv.metadata import Metadata
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer

In [38]:
def get_data_from_model(model_path, num_rows=1000, condition_dict=None):
    print(f"Loading model from: {model_path}")

    if "GaussianCopula" in model_path:
        model = GaussianCopulaSynthesizer.load(model_path)
    elif "CTGAN" in model_path:
        model = CTGANSynthesizer.load(model_path)
    else:
        raise ValueError(f"Cannot determine model type from file name: {model_path}")

    if condition_dict:
        print(f"Sampling with conditions: {condition_dict}")
        return model.sample_conditions(conditions=condition_dict, num_rows=num_rows)
    else:
        return model.sample(num_rows=num_rows)

In [41]:
def data_sythesizer(args, input_df=pd.DataFrame()):
    """Synthesize input dataframe data and return synthetic output."""

    # === Step 1: 若有 primary key，先轉為 str 避免 regex 檢查錯誤 ===
    pri_key = args.primary_key
    if pri_key and pri_key in input_df.columns:
        input_df[pri_key] = input_df[pri_key].astype(str)

    # === Step 2: 建構 metadata ===
    # metadata = Metadata.detect_from_dataframe(data=input_df)
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=input_df)

    if pri_key:
        metadata.update_column(column_name=pri_key, sdtype="id")
        metadata.set_primary_key(pri_key)
        print(f"Primary key '{pri_key}' set as sdtype='id'.")

    # === Step 3: Initialize model ===
    if args.synth_model == "GaussianCopula":
        print("Synthetic model arch: GaussianCopula")
        model = GaussianCopulaSynthesizer(metadata)
    elif args.synth_model == "CTGAN":
        if args.custom_setting:
            print("Synthetic model arch: CTGAN (custom)")
            model = CTGANSynthesizer(
                metadata=metadata,
                epochs=args.epochs,
                batch_size=args.batch_size,
                generator_dim=tuple(args.gen_dim),
                discriminator_dim=tuple(args.dis_dim),
                verbose=True,
            )
        else:
            logging.info("Synthetic model arch: CTGAN (default)")
            model = CTGANSynthesizer(metadata, verbose=True)
    else:
        raise ValueError(f"Unsupported synth_model: {args.synth_model}")

    # === Step 4: Fit model ===
    print("Fitting synthetic model ...")
    start_time = time.time()
    model.fit(input_df)
    print(f"Training completed in {time.time() - start_time:.2f} seconds")

    # === Step 5: Sample data ===
    output_df = model.sample(num_rows=args.num_rows)

    # === Step 6: Save model if specified ===
    if args.save_model:
        print("=== save Syn. Model file ===")
        model_name = f"syn_model_{args.synth_model}"
        model_name += ".pkl"

        output_dir = os.path.dirname(args.output_fpath) if args.output_fpath else args.output_dir
        os.makedirs(output_dir, exist_ok=True)
        model_path = os.path.join(output_dir, model_name)

        print(f"Saving model to {model_path}")
        model.save(model_path)

    # === Step 7: Save output CSV if specified ===
    if args.save_output and args.output_fpath:
        os.makedirs(os.path.dirname(args.output_fpath), exist_ok=True)
        print(f"Saving synthetic output to {args.output_fpath}")
        output_df.to_csv(args.output_fpath, index=False)

    return output_df

In [29]:
def set_args(args_list=None):
    """Main Function
    process input and do configs check
    """
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--input_path", type=str, default="data/train.csv")
    parser.add_argument("--output_dir", type=str, default="data/output/")
    parser.add_argument("--synth_model", type=str, default="GaussianCopula", help="sythetic model type")
    parser.add_argument("--primary_key", type=str, default="", help="primary key in your tabular data")
    parser.add_argument("--num_rows", type=int, default=200, help="num rows of the output sythetic dataframe")
    parser.add_argument("--save_model", action="store_true", help="set for save model pkl file")
    parser.add_argument("--save_output", action="store_true", help="set for save output csv file")
    parser.add_argument("--save_report", action="store_true", help="set for save report csv and image files")
    
    parser.add_argument("--custom_setting", action="store_true", help="set for custom setting in CTGAN and TVAE Model")
    parser.add_argument("--epochs", type=int, default=300, help="set epochs for training CTGAN and TVAE Model")
    parser.add_argument("--batch_size", type=int, default=500, help="set batch size for training CTGAN and TVAE Model")
    parser.add_argument("--gen_dim", type=int, nargs="+", default=[256, 256], help="set gen dimension")
    parser.add_argument("--dis_dim", type=int, nargs="+", default=[256, 256], help="set dis dimension")
    
    parser.add_argument("--input_syn_model", type=str, default=None, help="path to your syn_data model file")
    parser.add_argument("--sample_condition", type=str, default=None, help="path to your syn_data sample condition json file")
    parser.add_argument("--output_fpath", type=str, default=None, help="set full file path for your syn_data output csv")

    return parser.parse_args(args_list)

In [30]:
def main(args):
    # logging.info(f"contents of args.primary_key {args.primary_key}")
    # logging.info(f"contents of args.custom_setting {args.custom_setting}")
    # logging.info(f"contents of args.gen_dim {args.gen_dim}")
    # logging.info(f"contents of args.dis_dim {args.dis_dim}")
    if not args.input_syn_model:
        print("=== Train Synthetic Model and Generate Sample SynData csv ===")
        assert os.path.exists(args.input_path), f"Can't find the input file at {args.input_path}."
        assert os.path.exists(args.output_dir), f"Can't find the output folder at {args.output_dir}."
        assert args.synth_model in ["GaussianCopula", "CTGAN"]
        # ["GaussianCopula", "CTGAN", "CopulaGAN", "TVAE"]

        
        input_path = args.input_path
        output_dir = args.output_dir
        # output_fname=args.output_fname

        print(f"input file path: {input_path}")
        print(f"output directory: {output_dir}")

        input_df = pd.read_csv(input_path)
        if args.primary_key:
            assert args.primary_key in input_df.columns

        # if "Id" in input_df.columns:
        #     input_df = input_df.drop(columns=["Id"])

        output_df = data_sythesizer(args=args, input_df=input_df)

        logging.info("output dataframe shape")
        logging.info(output_df.shape)
        logging.info("output dataframe head(5)")
        logging.info(output_df.head())


        if args.save_output:
            print("=== save output csv file ===")
            
            base = os.path.basename(input_path)
            output_fname = (
                os.path.splitext(base)[0] + "_" + args.synth_model + "_output.csv"
            )
            output_df.to_csv(os.path.join(output_dir, output_fname), index=False)
            
            print(f"saved to {os.path.join(output_dir, output_fname)}")

    else:
        print("=== Generate Synthetic Data from Syn. Model ===")
        model_path = args.input_syn_model
        num_rows = args.num_rows
        condition_fpath = args.sample_condition
        output_fpath=args.output_fpath

        print(f"The Syn. Model Path: {model_path}")
        print(f"Generate {num_rows} rows to {output_fpath}...")
        
        assert os.path.exists(model_path), f"Can't find the model_path pkl file: {model_path}."
        if not condition_fpath:
            output_df = get_data_from_model(model_path, num_rows=num_rows, condition_dict=None)
        else:
            assert os.path.exists(condition_fpath), f"Can't find the sample_condition json file: {condition_fpath}."
            assert condition_fpath[-5:] == ".json", f"{condition_fpath} must be a json file!"
            
            with open(condition_fpath, "r") as f:
                condition_dict = json.load(f)

            output_df = get_data_from_model(model_path, num_rows=num_rows, condition_dict=condition_dict)

        print("output dataframe shape")
        print(output_df.shape)
        print("output dataframe head(5)")
        print(output_df.head())

        if args.save_output:
            print(f"=== Save csv to {output_fpath} ===")
            output_df.to_csv(output_fpath, index=False)


# 使用模組訓練合成資料模型 + 生成合成資料

In [31]:
args = set_args([
    "--input_path", "input/data.csv", # input 真實資料 csv 格式的表單
    "--output_dir", "output/",   # 合成資料輸出路徑
    "--synth_model", "GaussianCopula",  # 合成資料模型演算法
    "--primary_key", "Id",  # key 欄位名稱
    "--num_rows", "100",  # 生成的資料筆數
    "--save_model",    # 設定模型
    "--save_output"    # 設定儲存
])

main(args)

=== Train Synthetic Model and Generate Sample SynData csv ===
input file path: input/data.csv
output directory: output/
Synthetic model arch: GaussianCopula
Fitting synthetic model ...


/home/ubuntu/prjt-synthetic-data/py39/venv/lib/python3.9/site-packages/sdv/metadata/single_table.py:835: UserWarning: There is an existing primary key 'Id'. This key will be removed.
  warnings.warn(
/home/ubuntu/prjt-synthetic-data/py39/venv/lib/python3.9/site-packages/sdv/single_table/base.py:162: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/home/ubuntu/prjt-synthetic-data/py39/venv/lib/python3.9/site-packages/sdv/single_table/base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Training completed in 4.36 seconds
=== save Syn. Model file ===
Saving model to output/syn_model_GaussianCopula.pkl
=== save output csv file ===
saved to output/data_GaussianCopula_output.csv


## 檢視真實資料與合成資料表單

In [32]:
import pandas as pd

real_data_df = pd.read_csv("input/data.csv")  # 真實資料路徑
syn_data_df = pd.read_csv("output/data_GaussianCopula_output.csv")  # 合成資料預設檔名為: 真實資料檔名 + "_GaussianCopula_output"

In [33]:
real_data_df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1118,20,RL,57.0,9764,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,130000
1,903,60,RL,63.0,7875,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,180000
2,658,70,RL,60.0,7200,Pave,NaN,Reg,HLS,AllPub,...,0,NaN,MnPrv,NaN,0,2,2008,WD,Normal,149000
3,339,20,RL,91.0,14145,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,400,5,2006,WD,Normal,202500
4,341,60,RL,85.0,14191,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,202900
5,553,20,RL,87.0,11146,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,255500
6,1122,20,RL,84.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,New,Partial,212900
7,1116,20,RL,93.0,12085,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,New,Partial,318000
8,1433,30,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,64500
9,489,190,RL,60.0,10800,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,ConLD,Normal,160000


In [34]:
syn_data_df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,sdv-id-gxCvBj,162,RL,63.0,14360,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,3023,6,2006,WD,Alloca,92953
1,sdv-id-dVpfPA,66,RL,86.0,14885,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdWo,NaN,1560,6,2006,WD,Normal,203210
2,sdv-id-NJmWTR,134,RL,66.0,9908,Grvl,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,260,3,2007,WD,Abnorml,392372
3,sdv-id-ztMExL,108,RL,60.0,5864,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,5908,9,2008,New,Normal,134682
4,sdv-id-CLWLTq,46,RL,120.0,13014,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,10543,7,2007,WD,Normal,198611
5,sdv-id-roiJjM,39,RL,66.0,6075,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdWo,NaN,8308,9,2009,WD,Normal,107509
6,sdv-id-ajbfMu,55,RL,NaN,13465,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,7489,5,2010,WD,Abnorml,135027
7,sdv-id-BObTCv,94,RL,57.0,14070,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,203,8,2007,WD,Alloca,203394
8,sdv-id-pySEgy,175,RM,38.0,14374,Pave,NaN,IR1,Bnk,AllPub,...,0,NaN,NaN,NaN,1457,10,2009,WD,Normal,123195
9,sdv-id-vwBOim,170,RL,41.0,5485,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,98,9,2006,WD,Normal,90538
